In [ ]:
function ha_baseline(grid_a;grid_z,P,chute, γ = 3,β = 0.96, max_iter = 1000, r = 0.04)

    # Primeiro passo dessa função é salvar o tamanho dos grids de capitais e de estados
    δ = (1 - β)/β
    na, nz =length(grid_a),length(grid_z)
    # Num Array de três dimensões, será posto o consumo dos agentes, sendo a terceira dimensão para cada k_linha possível
    c = zeros(na, nz,na)
    # O consumo é calculado nesse loop
    for i_a in 1:na
        for i_z in 1:nz
            a = grid_a[i_a]
            z = grid_z[i_z]
            c[i_a,i_z,:] = z .+ (1+r)*a .- grid_a 
        end
    end

    # O consumo encontrado na etapa anterior é então aplicado na função utilidade caso seja positivo
    u = replace(x -> x>0 ? (x^(1-γ) - 1)/(1-γ) : -999999999,c)    

    # definimos v0 como o chute inicial, posto como input na função construída
    v0 = chute 

    v1 = zeros(na,nz) # função valor futuro
    pk = ones(na,nz) # função política do capital
    pc = zeros(na,nz)

    erro = 10
    tol = 1e-6

    for b in 1:max_iter # começo da iteração
        if erro > tol
            for i in 1:na # rodada de capital
                v = u[i,:,:] .+ β.*P*v0' #Matriz dos possíveis valores para um determinado capital
                # função política para cada estado
                estados = argmax(v,dims = 2)[:] 
                for s in 1:nz
                    a_linha = estados[s][2]
                    v1[i,s] = u[i,s,a_linha] + β*v0[a_linha,:]'*P[s,:]
                    pk[i,s] = a_linha
                end # rodada dos estados
            end # rodada dos capitais
            erro = maximum(abs.(v1 - v0))
            #println("iter ", b, " erro ", erro)
            v0 = copy(v1)
        else
            break
        end # if depois else
    end # iteração máxima
    #política de consumo
    # pc = grid_z' .+ (1+r).*grid_a .- grid_a[Int.(pk)]
    for i_a in 1:na
        for i_z in 1:nz
            pc[i_a,i_z] = c[i_a,i_z,Int.(pk[i_a,i_z])]
        end
    end
    return v1, pk, pc
end

In [ ]:
function ha_acelerado(grid_a;grid_z,P,chute, γ = 3,β = 0.96, max_iter = 1000, r = 0.04)

    # Primeiro passo dessa função é salvar o tamanho dos grids de capitais e de estados
    δ = (1 - β)/β
    na, nz =length(grid_a),length(grid_z)
    # Num Array de três dimensões, será posto o consumo dos agentes, sendo a terceira dimensão para cada k_linha possível
    c = zeros(na, nz,na)
    # O consumo é calculado nesse loop
    for i_a in 1:na
        for i_z in 1:nz
            a = grid_a[i_a]
            z = grid_z[i_z]
            c[i_a,i_z,:] = z .+ (1+r)*a .- grid_a 
        end
    end

    # O consumo encontrado na etapa anterior é então aplicado na função utilidade caso seja positivo
    u = replace(x -> x>0 ? (x^(1-γ) - 1)/(1-γ) : -999999999,c)    

    # definimos v0 como o chute inicial, posto como input na função construída
    v0 = chute 

    v1 = zeros(na,nz) # função valor futuro
    pk = ones(na,nz) # função política do capital
    pc = zeros(na,nz)

    erro = 10
    tol = 1e-6

    for b in 1:max_iter # começo da iteração
        if erro > tol || b % 10 != 0
            if b == 1 || b% 10 ==0
                for i in 1:na # rodada de capital
                    v = u[i,:,:] .+ β.*P*v0'
                    pk[i,:] = getindex.(argmax(v,dims = 2),2)
                    
                    for s in 1:nz
                        a = Int(pk[i,s])
                        v1[i,s] = u[i,s,a] + β*v0[a,:]'*P[s,:]
                    end# rodada dos estados
                end # rodada dos capitais
                erro = maximum(abs.(v1 - v0))
                #println("iter ", b, " erro ", erro)
                v0 = copy(v1)
            else
                for i in 1:na # rodada de capital
                    for s in 1:nz
                        a = Int(pk[i,s])
                        v1[i,s] = u[i,s,a] + β*v0[a,:]'*P[s,:]
                    end# rodada dos estados
                end # rodada dos capitais
                erro = maximum(abs.(v1 - v0))
                #println("iter ", b, " erro ", erro)
                v0 = copy(v1)
            end
        else
            break
        end # if depois else
    end # iteração máxima
    #política de consumo
    # pc = grid_z' .+ (1+r).*grid_a .- grid_a[Int.(pk)]
    for i_a in 1:na
        for i_z in 1:nz
            pc[i_a,i_z] = c[i_a,i_z,Int.(pk[i_a,i_z])]
        end
    end
    return v1, pk, pc
end

In [ ]:
function ilv(grids;m,v2,v1,K2,K1,nz)
    r = grids[m]/grids[m-1]
    for is in 1:nz
        for ik in 1:grids[m]
            if ik == 1
                v2[ik,is] = v1[ik,is]
            elseif ik == grids[m]
                v2[ik,is] = v1[grids[m-1],is]
            else
                a = Int(div(ik-1,r) + 1)
                b = Int(div(ik-1,r) + 2)
                if a == grids[m-1]
                    v2[ik,is] = v1[a-1,is] + ((v1[b-1,is] - v1[a-1,is])/(K1[b-1] - K1[a-1]))*(K2[ik]-K1[a-1])
                else
                    v2[ik,is] = v1[a,is] + ((v1[b,is] - v1[a,is])/(K1[b] - K1[a]))*(K2[ik]-K1[a])
                end
            end
        end
    end 
    return v2
end

In [ ]:
function ha_multigrid(grids;grid_z,P,β = 0.96,γ = 3,r = 0.04)
    M = length(grids)
    nz = length(grid_z)
    pk = Array{Int32}
    v1 = Array{Float32}
    km = Array{Float32}
    vm = Array{Float32}
    c1 = Array{Float32}
    ϕ = -minimum(grid_z)/(1/β)
    a_max = 4*maximum(grid_z)/(1/β)
    for m in 1:M # quantidade de grids a serem usados
        if m == 1
            vm = zeros(grids[m],nz)
            km = LinRange(ϕ,a_max,grids[m])
            v1, pk, c1= ha_baseline(km;grid_z,P,chute = vm, γ = γ,β = β, max_iter = 1000, r = r)
        else
            kn = LinRange(ϕ,a_max,grids[m])
            vn = zeros(grids[m],nz)
            vm = ilv(grids;m=m,v2=vn, v1=v1, K2 = kn,K1 = km,nz = nz)
            v1, pk, c1 = ha_baseline(kn;grid_z,P,chute = vn,γ = γ,β = β, max_iter = 1000, r = r)
            km = LinRange(ϕ,a_max,grids[m])
        end
        
    end #qtde de grids
    return v1, pk, c1
end

In [ ]:
function ha_mg_acelerado(grids;grid_z,P,β = 0.96,γ = 3,r = 0.04)
    M = length(grids)
    nz = length(grid_z)
    pk = Array{Int32}
    v1 = Array{Float32}
    km = Array{Float32}
    vm = Array{Float32}
    c1 = Array{Float32}
    ϕ = -minimum(grid_z)/(1/β)
    a_max = 4*maximum(grid_z)/(1/β)
    for m in 1:M # quantidade de grids a serem usados
        if m == 1
            vm = zeros(grids[m],nz)
            km = LinRange(ϕ,a_max,grids[m])
            v1, pk, c1= ha_baseline(km;grid_z,P,chute = vm, γ = γ,β = β, max_iter = 1000, r = r)
        else
            kn = LinRange(ϕ,a_max,grids[m])
            vn = zeros(grids[m],nz)
            vm = ilv(grids;m=m,v2=vn, v1=v1, K2 = kn,K1 = km,nz = nz)
            v1, pk, c1 = ha_acelerado(kn;grid_z,P,chute = vn,γ = γ,β = β, max_iter = 1000, r = r)
            km = LinRange(ϕ,a_max,grids[m])
        end
        
    end #qtde de grids
    return v1, pk, c1
end